### Objectives

First, create a graph representation of the information we have. 
This would be information about the elements, and the nodes. 
Using this, we will feed it into our SVM, or a Random Forest, and see from there onwards

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import numpy as np
import ast
import io


# Set up torch to use CUDA or MPS if either are available
device = torch.device('cuda:0')  if torch.cuda.is_available() else torch.device('mps:0') if torch.backends.mps.is_available() else torch.device('cpu')
torch.set_default_device(device)

USE_NEW_FEATURES = True
TRAIN_SVM = False
TRAIN_RF = False

MLP_MODEL_PATH = './mlp_regressor.pth'
LOAD_MLP_MODEL = False
SAVE_MLP_MODEL = True

In [ ]:
n = pd.read_csv('./full_node_Table_.csv')
df = pd.read_csv('./full_Table_.csv')

In [ ]:
# cc_df = df.sample(n=1000, random_state=1)
cc_df = df
cc_df['cc_pixel_intensity_488'] = cc_df['cc_pixel_intensity_488'].astype(int)

X = cc_df.drop(['cc_pixel_intensity_488', 'cc_pixel_intensity_405', 'cc_x', 'folder_name_x', 'degree_distribution', 'folder_name_x.1', 'folder_name_y', 'x_y', 'y_y', 'z_y', 'node', 'degree', 'cc_y', 'vol_cc', 'avg_PK_Of_element', 'element_connectivity', 'cc_average_connectivity', 'pixel_intensity_488',
'pixel_intensity_405',
'pixel_intensity_ratio',
'pixel_intensity_555',
'element_pixel_intensity_488',
'element_pixel_intensity_405',
'element_pixel_intensity_ratio'], axis=1)  # Adjust 'target_column' to your actual target column name
# Print rows that have NaN in any column, and print that column name
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

# For any rows that contain a NaN in X, drop it in X and drop the corresponding row in Y.
# This is done by getting the row index where X is NaN and dropping the same index in both X and Y
nan_rows = X.isna().any(axis=1)
X = X.dropna()

y = cc_df['cc_pixel_intensity_488']
y = y[~nan_rows]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = y_train.to_numpy()

for x in X_train.columns:
    print(x)

# y_test=y_test.to_numpy()

In [ ]:
datafrac = 1
# This cell reads the data from the CSV file and cleans it up

if(USE_NEW_FEATURES):

    # Set pandas options to display all columns and rows
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    csv_file_path = 'full_Table_All_Cells_With_Angles_And_Edges_CLEANED.csv'
    buffer = io.StringIO()

    # Open and process the CSV file, to strip entries so that numbers aren't read as strings by read_csv and column names have no leading/trailing whitespaces
    with open(csv_file_path, 'r') as file:
        for line in file:
            cleaned_line = ','.join(cell.strip() for cell in line.split(','))
            buffer.write(cleaned_line + '\n')

    # Move the buffer cursor to the start
    buffer.seek(0)

    # Read the cleaned data into pandas
    data = pd.read_csv(buffer)
    edge_columns = [col for col in data.columns if col.startswith('edge_') and col.split('_')[-1].isdigit()]
    # Replace empty entries with 0 in these columns
    data[edge_columns] = data[edge_columns].fillna(0)

    # Not sure if these folder names are required for segregating the dataset, so for now dropping them
    data = data.drop(columns=['folder_name_x', 'folder_name_x.1', 'folder_name_y'])
    # For now, we aren't using the 555 channel either
    data = data.drop(columns=['element_pixel_intensity_555'])

    # These fields seem to be not implemented either:
    data = data.drop(columns=["Unnamed: 0_x", "x", "y" ,"z" ,"node_x" ,"degree_x" ,"vol_cc_x" ,"avg_PK_Of_element_x" ,"element_connectivity_x" ,"Unnamed: 0_y"])

    # The next few lines of code are to convert the degree_distribution column to columns labelled node_degree_0, node_degree_1, etc. till node_degree_MAX
    def str_to_dict(s):
        try:
            return ast.literal_eval(s)
        except ValueError:
            return {}
    # Apply the function to the 'degree_distribution' column
    data['degree_distribution'] = data['degree_distribution'].apply(str_to_dict)
        
    # Find the highest node degree in the DataFrame
    max_degree = max(degree for row in data['degree_distribution'] for degree in row.keys())

    # Create a DataFrame for new columns
    new_columns = pd.DataFrame({f'node_degree_{i}': 0 for i in range(1, max_degree + 1)}, index=data.index)

    # Concatenate with the original DataFrame
    data = pd.concat([data, new_columns], axis=1)
        
    # Function to update the degree columns for a row
    def update_degrees(row):
        for degree, count in row['degree_distribution'].items():
            row[f'node_degree_{degree}'] = count
        return row

    # Apply the function to each row
    data = data.apply(update_degrees, axis=1)
    data = data.drop(columns=['degree_distribution'])
    # Now we should have columns labelled node_degree_0, node_degree_1, etc. till node_degree_MAX and dropped the degree_distribution column

    non_numerical_columns = data.select_dtypes(exclude=['int', 'float', 'double']).columns
    # Print the non-numerical column names
    if(len(non_numerical_columns) > 0):
        print(non_numerical_columns)
        raise Exception('Non-numerical columns found in dataset, can\'t train model on it directly')

    data['cc_average_connectivity'] = data['cc_average_connectivity'].replace([np.inf, -np.inf], 0)

    # Print any columns with values like infinity
    columns_with_infinity = data.columns.to_series()[data.apply(lambda x: np.isinf(x).any())]
    if(len(columns_with_infinity) > 0):
        print(columns_with_infinity)
        raise Exception('Columns with infinity values found in dataset, can\'t train model on it directly')

    # Drop rows that have NaN values in columns 'clustering coefficient', 'pixel_intensity_ratio', 'vol_cc_y', 'x_y', 'y_y', 'z_y'
    # data = data.dropna(subset=['clustering coefficient', 'pixel_intensity_ratio', 'vol_cc_y', 'x_y', 'y_y', 'z_y'])

    # Replace any remaining NaN's with 0
    data = data.fillna(0)

    columns_with_NaN = data.columns.to_series()[data.apply(lambda x: np.isnan(x).any())]
    if(len(columns_with_NaN) > 0):
        print(columns_with_NaN.values)
        raise Exception('Columns with NaN values found in dataset, can\'t train model on it directly')

    # Train on small subset of data to see if it works first
    sample_data = data.sample(frac=datafrac ,random_state=1049)

    # Split the data into training and testing sets
    X = sample_data.drop(columns=['element_pixel_intensity_488'])
    X = X.drop(['cc_pixel_intensity_488',
                'cc_pixel_intensity_405',
                'cc_x', 'x_y', 'y_y', 'z_y', 'cc_y',
                'pixel_intensity_405',
                'pixel_intensity_ratio',
                # 'pixel_intensity_555',
                'element_pixel_intensity_405',
                'element_pixel_intensity_ratio',
                'cc_average_connectivity',
                # 'cc_pixel_intensity_555',
                'cc_pixel_intensity_ratio', 
                'pixel_intensity',
                'pixel_intensity_488'], axis=1)
    
    xlabels = X.columns
    with open("training_feature_list_excl_angles_and_edges.txt", "w") as file:
        for thing in xlabels:
            if "Angle" not in thing and "edge" not in thing:
                file.write(thing + "\n")
    
    sample_data['element_pixel_intensity_405_488_ratio'] = (np.array(sample_data['element_pixel_intensity_405']) / np.array(sample_data['element_pixel_intensity_488']))*1000
    sample_data.fillna(0, inplace=True)
    
    # We're basically predicting
    # (405 / 488) * 1000

    Y = sample_data['element_pixel_intensity_405_488_ratio']
    # Group all Y values into classes of 50
    # Y = np.floor(Y / 50) * 50

    # Perform standardization on X and Y
    scalerX = StandardScaler()
    scalerY = StandardScaler()

    # X = scalerX.fit_transform(X)
    # Y = scalerY.fit_transform(Y.values.reshape(-1, 1))
    y = np.rint(Y).astype(int)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # y_train, y_test = y_train.reshape(-1), y_test.reshape(-1)
    print("The training data shapes are:", X_train.shape, y_train.shape, "and the testing data shapes are:", X_test.shape, y_test.shape)

In [ ]:
def plot_error_frequency(y_pred, y_test, classifier_title, rounding):
    rounding_ = rounding//2
    
    # Calculate errors
    errors = (abs(y_pred - y_test) < rounding_)

    # Extract labels where errors occurred
    error_labels = y_test[errors]

    # Calculate frequency of each label
    unique_labels, counts = np.unique(error_labels, return_counts=True)

    # Creating a dictionary of label frequencies for errors
    label_error_frequency = dict(zip(unique_labels, counts))

    # Finding the range of all possible labels (from min to max in y_test and y_pred)
    all_labels = np.unique(np.concatenate((y_test, y_pred)))
    min_label = min(all_labels)
    max_label = max(all_labels)

    # Filling in frequencies for labels that did not have errors
    for label in range(min_label, max_label + 1):
        if label not in label_error_frequency:
            label_error_frequency[label] = 0

    # Sorting the dictionary for plotting
    sorted_label_error_frequency = dict(sorted(label_error_frequency.items()))

    xticks = np.arange(min_label, max_label + 1, 50)

    # Plotting the error frequency
    plt.figure(figsize=(10, 6))
    plt.bar(sorted_label_error_frequency.keys(), sorted_label_error_frequency.values())
    plt.xlabel('Prediction')
    plt.ylabel('Error Frequency')
    plt.title('Error Frequency Distribution for ' + str(classifier_title) + ' for errors >= {} ({} in either direction)'.format(rounding, rounding_))
    plt.xticks(xticks, rotation='vertical')  # Set x-ticks to show every 50th label and rotate them vertically
    plt.show()

    # Calculating mean and standard deviation of absolute errors
    absolute_errors = np.abs(y_pred - y_test)
    mean_error = np.mean(absolute_errors)
    std_error = np.std(absolute_errors)
    print("The mean error is", mean_error, "and the standard deviation of the errors is", std_error)

    # Plotting absolute error for each label
    plt.figure(figsize=(10, 6))
    for label in all_labels:
        label_errors = absolute_errors[y_test == label]
        plt.bar([label]*len(label_errors), label_errors, color='r')
    plt.xlabel('Prediction')
    plt.ylabel('Absolute Error')
    plt.title('Absolute Error for Each Label in ' + str(classifier_title))
    plt.xticks(xticks, rotation='vertical')
    plt.show()


In [ ]:
# For y_train and y_test, round down to nearest 10
# y_train = (np.floor(y_train / 10) * 10).astype(int)
# y_test = (np.floor(y_test / 10) * 10).astype(int)

# Noticed weird behaviour with SVM:
# If y_train and y_test are not rounded down to nearest 10,
# then the training takes LONGER (it just wouldn't finish training on my mac)
# even though it means there are 10 times fewer classifiers to train?
# (also yes, doing that means the output would not be a regression but a classification,
# and outputs would be classified into groups of 10, as in the model would tell you whether the
# functional parameter is between 0-10, 10-20, 20-30, etc.)

# Also it was not possible to get the SVM to train on my MAC for datasets above 10k rows (10k including training and testing split)
# Training just never finished if I tried that, even if I left it running overnight.

# This makes sense though because with more classes (i.e discrete functional parameter values in this case),
# the training time grows quadratically as that many more classifiers need to be trained.

# print(y_train, y_test)

if(TRAIN_SVM):

    svm_model = SVC(kernel='linear')  # You can choose different kernels like 'rbf', 'poly', etc.
    svm_model.fit(X_train, y_train)


    y_pred = svm_model.predict(X_test)

    print(pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}))

    accuracy = accuracy_score(y_test, y_pred)
    classification_rep = classification_report(y_test, y_pred)

    plt.plot(y_test, y_pred, 'o')
    plt.title('Accuracy: ' + str(accuracy))
    plt.xlabel('Actual')
    plt.ylabel('Predicted')

    y_pred_svm = y_pred



In [ ]:
if(TRAIN_SVM):
    plot_error_frequency(y_pred_svm, y_test, 'SVC')

In [ ]:
if(TRAIN_RF):

    # do the same for random forest
    from sklearn.ensemble import RandomForestClassifier

    rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, random_state=0)
    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    plt.plot(y_test, y_pred, 'o')
    plt.title('Accuracy: {}'.format(accuracy))

    y_pred_rf = y_pred

In [ ]:
if(TRAIN_RF):
    plot_error_frequency(y_pred_rf, y_test, 'Random Forest')

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Used the plotly library as this is much nicer looking, and the graph is more interactive and works well with Jupyter Notebooks

def plot_absolute_error_plotly(y_pred_model1, y_pred_model2, y_test, model_title1, model_title_2):
    def calculate_error_frequency(y_pred):
        incorrect = y_pred != y_test

        unique_predictions, counts = np.unique(y_pred[incorrect], return_counts=True)
        error_freq = dict(zip(unique_predictions, counts))

        return error_freq

    error_freq_svc = calculate_error_frequency(y_pred_model1)
    error_freq_rf = calculate_error_frequency(y_pred_model2)

    all_labels = np.unique(np.concatenate((y_pred_model1, y_pred_model2)))
    all_labels.sort()

    svc_freq = [error_freq_svc.get(label, 0) for label in all_labels]
    rf_freq = [error_freq_rf.get(label, 0) for label in all_labels]

    fig = go.Figure(data=[
        go.Bar(name=model_title1, x=all_labels, y=svc_freq),
        go.Bar(name=model_title_2, x=all_labels, y=rf_freq)
    ])

    fig.update_layout(
        barmode='group',
        title='Prediction Error Frequency',
        xaxis_title='Predicted Label',
        yaxis_title='Frequency of Incorrect Predictions',
        xaxis=dict(type='category')
    )

    fig.show()


def plot_error_label_frequency_plotly(y_pred_model1, y_pred_model2, y_test, model_title1, model_title2):
    labels = np.unique(y_test)
    errors_svc = [np.sum((y_pred_model1 != y_test) & (y_test == label)) for label in labels]
    errors_rf = [np.sum((y_pred_model2 != y_test) & (y_test == label)) for label in labels]

    bar_width = max(0.5, 30 / len(labels))

    fig = go.Figure(data=[
        go.Bar(name=model_title1, x=labels, y=errors_svc, width=bar_width, marker_color='blue', marker_line_color='blue', marker_line_width=1.5, opacity=1),
        go.Bar(name=model_title2, x=labels, y=errors_rf, width=bar_width, marker_color='red', marker_line_color='red', marker_line_width=1.5, opacity=1)
    ])
    
    fig.update_layout(
        barmode='group',
        title='Error Frequency per Label',
        xaxis_title='Labels',
        yaxis_title='Number of Errors',
        xaxis=dict(tickmode='array', tickvals=labels[::10]),
        plot_bgcolor='rgba(255, 255, 255, 1)',
        font=dict(
            size=12,  # You can adjust the size of the text here for better visibility
        )
    )
    fig.show()


In [ ]:
if(TRAIN_RF and TRAIN_SVM):
    plot_absolute_error_plotly(y_pred_svm, y_pred_rf, y_test, 'SVC', 'Random Forest')
    plot_error_label_frequency_plotly(y_pred_svm, y_pred_rf, y_test, 'SVC', 'Random Forest')

In [ ]:
# Data distribution
# Basically plot the frequency of ocurrence of each value
# of the functional parameter in the dataset inputted to the models

# This is only plotting the subset in the training set, and not in the entire dataset

def plot_data_distribution(y):
    # Calculate the frequency of each value in the y_test array
    unique_values, counts = np.unique(y, return_counts=True)

    # Create the bar chart
    fig = go.Figure(data=go.Bar(x=unique_values, y=counts))

    # Update the layout for a better visual representation
    fig.update_layout(
        title='Data Distribution',
        xaxis_title='Unique Values',
        yaxis_title='Frequency',
        xaxis=dict(type='category'),  # Treat unique values as discrete categories
        yaxis=dict(title='Frequency'),  # You can also set the range or scale of the y-axis if needed
    )

    fig.show()
    print("The mean of the data distribution is", np.mean(y_train), "and the standard deviation is", np.std(y_train))

In [ ]:

plot_data_distribution(y)

In [ ]:
# Define the MLP regressor class
class MLPRegressor(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size//2)
        self.fc3 = nn.Linear(hidden_size//2, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the MLP regressor
input_size = X_train.shape[1]
# hidden_size = int((X_train.shape[1] + 1) / 2)
hidden_size = input_size
mlp_regressor = MLPRegressor(input_size, hidden_size).to(device)
if(LOAD_MLP_MODEL):
    try:
        mlp_regressor.load_state_dict(torch.load(MLP_MODEL_PATH))
    except Exception as e:
        print("Couldn't load model")
        print(e)

# Define the loss function and optimizer
criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(mlp_regressor.parameters(), lr=0.001, weight_decay=1e-5)

def create_batches(X, Y, batch_size):
    num_batches = int(np.ceil(X.shape[0] / batch_size))
    batches = []

    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size
        X_batch = X[start:end]
        Y_batch = Y[start:end]
        batches.append((X_batch, Y_batch))

    return batches

# Train the MLP regressor
def train_mlp_regressor(epochs, batch_size, X_train, Y_train):
    # Convert the training data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)

    # Create manual batches
    train_batches = create_batches(X_train_tensor, Y_train_tensor, batch_size)

    for epoch in range(epochs):
        for X_batch, Y_batch in train_batches:
            # Move batch to the device
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

            # Forward pass
            outputs = mlp_regressor(X_batch)
            loss = criterion(outputs, Y_batch)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

        
def test_mlp_regressor(X_test, Y_test):
    # Convert the testing data to PyTorch tensors
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32).squeeze()

    # Test the MLP regressor
    with torch.no_grad():
        outputs = mlp_regressor(X_test_tensor)
        loss = criterion(outputs, Y_test_tensor)
        print(f'Loss: {loss.item():.4f}')
        
    # Convert the predictions to a NumPy array
    return outputs.cpu().detach().numpy().squeeze()

In [ ]:
if(not LOAD_MLP_MODEL):
    train_mlp_regressor(epochs=1000, batch_size=500, X_train=X_train.values if isinstance(X_train, pd.DataFrame) else X_train, Y_train=np.array(y_train).reshape(-1, 1))
# Interrupt this cell whenever it feels like this model has trained enough

In [ ]:
# Save model to disk
if(SAVE_MLP_MODEL):
    torch.save(mlp_regressor.state_dict(), MLP_MODEL_PATH)

In [ ]:
def calculate_accuracy(y_test, y_pred, rounding):
    y_test = y_test.values
    rounding = rounding//2
    # print(y_pred)
    assert(len(y_test) == len(y_pred))
    num_predictions = len(y_test)
    num_correct_predictions = 0
    for i in range(len(y_test)):
        if abs(y_test[i] - y_pred[i]) < rounding:
            num_correct_predictions += 1
    
    return num_correct_predictions/num_predictions

In [ ]:

y_pred_mlp = test_mlp_regressor(X_test.values if isinstance(X_test, pd.DataFrame) else X_test, np.array(y_test).reshape(-1, 1).squeeze())
y_pred_mlp = np.rint(y_pred_mlp).astype(int)

rounding_graph = 1
rounding = 100

plt.plot(np.floor(y_test//rounding_graph) * rounding_graph, np.floor(y_pred_mlp//rounding_graph) * rounding_graph, 'o')
# accuracy = accuracy_score(np.floor(y_test//rounding) * rounding, np.floor(y_pred_mlp//rounding) * rounding)
accuracy = calculate_accuracy(y_test, y_pred_mlp, rounding)

plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Accuracy (prediction error < {}): {}'.format(rounding, accuracy))

In [ ]:

# y_pred_mlp = test_mlp_regressor(X_train.values if isinstance(X_train, pd.DataFrame) else X_train, np.array(y_train).reshape(-1, 1).squeeze())
# y_pred_mlp = np.rint(y_pred_mlp).astype(int)

# rounding = 10

# plt.plot(np.floor(y_train//rounding) * rounding, np.floor(y_pred_mlp//rounding) * rounding, 'o')
# accuracy = accuracy_score(np.floor(y_train//rounding) * rounding, np.floor(y_pred_mlp//rounding) * rounding)

# plt.xlabel('Actual')
# plt.ylabel('Predicted')
# plt.title('Accuracy (within nearest {}): {}'.format(rounding, accuracy))

In [ ]:
plot_error_frequency(y_pred_mlp, y_test, 'MLP', rounding)

In [ ]:
# Create a DataFrame with the test and pred values
df = pd.DataFrame({'Test': y_test, 'Pred': y_pred_mlp})

# Save the DataFrame to a CSV file
df.to_csv('mlp_test_pred.csv', index=False)